<a href="https://colab.research.google.com/github/Zerowaltz/dl-final-project/blob/main/dl_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
!pip install seqeval
!pip install tensorflow_text
#import tensorflow_text as text
import spacy
from spacy import displacy
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Concatenate
from tensorflow.keras.layers import add
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda
from tensorflow.keras.utils import plot_model
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
tf.compat.v1.enable_eager_execution()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Preprocessing Groningen Meaning Bank dataset:

In [ ]:


################################################################################
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
################################################################################

data = pd.read_csv("drive/MyDrive/ner_dataset.csv", encoding="latin1")
data = data.drop(['POS'], axis =1) #dont need parts of speech tags 
data = data.fillna(method="ffill")

words = set(list(data['Word'].values))
words.add('PADword')
n_words = len(words)

tags = list(set(data["Tag"].values))
n_tags = len(tags)

getter = SentenceGetter(data)
sentences = getter.sentences
largest_sen = max(len(sen) for sen in sentences)

words2index = {w:i for i,w in enumerate(words)}
tags2index = {t:i for i,t in enumerate(tags)}
print(tags)
print(tags2index)
data['Word_idx'] = data['Word'].map(words2index)
data['Tag_idx'] = data['Tag'].map(tags2index)



max_len = 50
X = [[w[0] for w in s] for s in sentences]
new_X = []
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("PADword")
    new_X.append(new_seq)
new_X[15]

y = [[tags2index[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tags2index["O"])

X_tr, X_te, y_tr, y_te = train_test_split(new_X, y, test_size=0.1, random_state=2021)


['B-tim', 'O', 'I-tim', 'B-geo', 'I-gpe', 'I-eve', 'B-eve', 'I-art', 'B-org', 'B-nat', 'I-per', 'I-nat', 'B-per', 'I-org', 'B-gpe', 'B-art', 'I-geo']
{'B-tim': 0, 'O': 1, 'I-tim': 2, 'B-geo': 3, 'I-gpe': 4, 'I-eve': 5, 'B-eve': 6, 'I-art': 7, 'B-org': 8, 'B-nat': 9, 'I-per': 10, 'I-nat': 11, 'B-per': 12, 'I-org': 13, 'B-gpe': 14, 'B-art': 15, 'I-geo': 16}


Defining, training, and testing model on GMB dataset:

In [ ]:

tf.compat.v1.disable_v2_behavior()

sess = tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session(sess)

batch_size = 32
X_tr, X_val = X_tr[:1213*batch_size], X_tr[-135*batch_size:]
y_tr, y_val = y_tr[:1213*batch_size], y_tr[-135*batch_size:]
y_tr = y_tr.reshape(y_tr.shape[0], y_tr.shape[1], 1)
y_val = y_val.reshape(y_val.shape[0], y_val.shape[1], 1)

elmo_model = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)
sess.run(tf.compat.v1.global_variables_initializer())
sess.run(tf.compat.v1.tables_initializer())

def ElmoEmbedding(x):
    return elmo_model(inputs={
                            "tokens": tf.squeeze(tf.cast(x, tf.string)),
                            "sequence_len": tf.constant(batch_size*[max_len])
                      },
                      signature="tokens",
                      as_dict=True)["elmo"]


input_text = Input(shape=(max_len,), dtype=tf.string)
embedding = Lambda(ElmoEmbedding, output_shape=(max_len, 1024))(input_text)
x = Bidirectional(LSTM(units=512, return_sequences=True,
                       dropout=0.5, activity_regularizer=tf.keras.regularizers.L2(1e-3)))(embedding)
x_rnn = Bidirectional(LSTM(units=512, return_sequences=True,
                           dropout=0.5, activity_regularizer=tf.keras.regularizers.L2(1e-3)))(x)
x = add([x, x_rnn])  # residual connection to the first biLSTM
out = TimeDistributed(Dense(n_tags, activation="softmax"))(x)

model = Model(input_text, out)
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

model.summary()

#plot_model(model,'Elmo_tag.png',show_shapes= True)
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
history = model.fit(np.array(X_tr), y_tr, validation_data=(np.array(X_val), y_val),
                    batch_size=batch_size, epochs=1, verbose=1, callbacks=[callback])
model.save("drive/MyDrive/dl_model")

X_te = X_te[:149*batch_size]
test_pred = model.predict(np.array(X_te), verbose=1)

idx2tag = {i: w for w, i in tags2index.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PADword", "O"))
        out.append(out_i)
    return out

def test2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            out_i.append(idx2tag[p].replace("PADword", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = test2label(y_te[:149*32])

print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))

print(classification_report(test_labels, pred_labels))

i = 395
p = model.predict(np.array(X_te[i:i+batch_size]))[0]
p = np.argmax(p, axis=-1)
print("{:15} {:5}: ({})".format("Word", "Pred", "True"))
print("="*30)
for w, true, pred in zip(X_te[i], y_te[i], p):
    if w != "PADword":
        print("{:15}:{:5} ({})".format(w, tags[pred], tags[true]))

s = ''
for i in X_te[395]:
  s+=i+' '
s = s.rstrip()


nlp = spacy.load('en_core_web_sm')
text = nlp(s)
displacy.render(text, style = 'ent', jupyter=True)

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 lambda (Lambda)                (32, None, 1024)     0           ['input_1[0][0]']                
                                                                                                  
 bidirectional (Bidirectional)  (32, None, 1024)     6295552     ['lambda[0][0]']                 
                                                                                                  
 bidirectional_1 (Bidirectional  (32, None, 1024)    6295552     ['bidirectional[0][0]']          
 )                                                                                            

/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


38816/38816 [==============================] - 454s 12ms/sample - loss: 0.1418 - acc: 0.9787 - val_loss: 0.0609 - val_acc: 0.9854


/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


F1-score: 81.4%


/usr/local/lib/python3.8/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         art       0.00      0.00      0.00        49
         eve       0.67      0.28      0.39        29
         geo       0.82      0.89      0.85      3682
         gpe       0.93      0.92      0.93      1637
         nat       0.50      0.18      0.26        17
         org       0.68      0.64      0.66      2022
         per       0.75      0.80      0.77      1687
         tim       0.87      0.84      0.86      2014

   micro avg       0.81      0.82      0.81     11137
   macro avg       0.65      0.57      0.59     11137
weighted avg       0.80      0.82      0.81     11137

Word            Pred : (True)
Mr.            :B-per (B-per)
Azhari         :I-per (I-per)
told           :O     (O)
VOA            :B-org (B-org)
reporter       :O     (O)
William        :B-per (B-org)
Eagle          :I-per (I-org)
that           :O     (O)
he             :O     (O)
expects        :O     (O)
the            :O     (O)
government   

Importing other datasets from Huggingface:

In [ ]:
!pip install datasets

from datasets import load_dataset



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#dataset = load_dataset("conll2003") #change what dataset is being used
dataset = load_dataset("wnut_17")
dataset_df_train = pd.DataFrame(dataset['train'][:])
dataset_df_train = dataset_df_train.explode(['tokens', 'ner_tags'])
dataset_df_val = pd.DataFrame(dataset['train'][:])
dataset_df_val = dataset_df_val.explode(['tokens', 'ner_tags'])
dataset_df_test = pd.DataFrame(dataset['train'][:])
dataset_df_test = dataset_df_test.explode(['tokens', 'ner_tags'])


Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #2:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split:   0%|          | 0/3394 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1287 [00:00<?, ? examples/s]

Dataset wnut_17 downloaded and prepared to /root/.cache/huggingface/datasets/wnut_17/wnut_17/1.0.0/077c7f08b8dbc800692e8c9186cdf3606d5849ab0e7be662e6135bb10eba54f9. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
import datasets
dataset['test']
all_data = datasets.concatenate_datasets([dataset['train'], dataset['test'], dataset['validation']])

Preprocessing imported dataset:

In [ ]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["tokens"].values.tolist(),s["ner_tags"].values.tolist())]
        self.grouped = self.data.groupby("id").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
################################################################################


words2 = set(np.concatenate([np.array(i) for i in all_data['tokens']], dtype=object))
words2.add('PADword')
n_words2 = len(words2)

tags2 = list(set(np.concatenate([np.array(i) for i in all_data['ner_tags']], dtype=np.int32)))
n_tags2 = len(tags2)
words2index2 = {w:i for i,w in enumerate(words2)}
tags2index2 = {t:i for i,t in enumerate(tags2)}
max_len = 50

def get_xy(dataframe):
  getter2 = SentenceGetter(dataframe)
  sentences2 = getter2.sentences
  largest_sen2 = max(len(sen) for sen in sentences2)

  words2index2 = {w:i for i,w in enumerate(words2)}

  #data['Word_idx'] = data['Word'].map(words2index)


  X_tr2 = [[w[0] for w in s] for s in sentences2]
  new_X = []
  for seq in X_tr2:
      new_seq = []
      for i in range(max_len):
          try:
              new_seq.append(seq[i])
          except:
              new_seq.append("PADword")
      new_X.append(new_seq)
  new_X[15]

  y_tr2 = [[w[1] for w in s] for s in sentences2]
  y_tr2 = pad_sequences(maxlen=max_len, sequences=y_tr2, padding="post", value=0)
  return new_X, y_tr2

Getting arrays to be the correct shape to pass through Elmo embedding

In [ ]:

tf.compat.v1.disable_v2_behavior()

sess2 = tf.compat.v1.Session()

tf.compat.v1.keras.backend.set_session(sess2)
X_tr, y_tr = get_xy(dataset_df_train)
X_te, y_te = get_xy(dataset_df_test)
X_val, y_val = get_xy(dataset_df_val)
batch_size = 10
X_tr = X_tr[:-(len(X_tr) % batch_size)]

y_tr = y_tr[:-(len(y_tr) % batch_size)]
X_val = X_val[:-(len(X_val) % batch_size)]
y_val = y_val[:-(len(y_val) % batch_size)]
#X_tr, X_val = X_tr[:1213*batch_size], X_tr[-135*batch_size:]
#y_tr, y_val = y_tr[:1213*batch_size], y_tr[-135*batch_size:]
y_tr = y_tr.reshape(y_tr.shape[0], y_tr.shape[1], 1)
y_val = y_val.reshape(y_val.shape[0], y_val.shape[1], 1)



Instructions for updating:
non-resource variables are not supported in the long term


Define ELMO embedding

In [ ]:
elmo_model = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)
sess2.run(tf.compat.v1.global_variables_initializer())
sess2.run(tf.compat.v1.tables_initializer())

def ElmoEmbedding(x):
    return elmo_model(inputs={
                            "tokens": tf.squeeze(tf.cast(x, tf.string)),
                            "sequence_len": tf.constant(batch_size*[max_len])
                      },
                      signature="tokens",
                      as_dict=True)["elmo"]

In [ ]:
sentence_encoder = hub.KerasLayer("https://tfhub.dev/google/sentence-t5/st5-base/1")


def SentEmbedding(x):
  return sentence_encoder(tf.squeeze(tf.cast(x, tf.string)))

Defining, training, and saving model to drive

In [ ]:
input_text = Input(shape=(max_len,), dtype=tf.string)

embedding = Lambda(ElmoEmbedding, output_shape=(max_len, 1024))(input_text)
x = Bidirectional(LSTM(units=512, return_sequences=True,
                       dropout=0.5, activity_regularizer=tf.keras.regularizers.L2(1e-3)))(embedding)
x_rnn = Bidirectional(LSTM(units=512, return_sequences=True,
                           dropout=0.5, activity_regularizer=tf.keras.regularizers.L2(1e-3)))(x)
wt = add([x, x_rnn])  # residual connection to the first biLSTM
#out = TimeDistributed(Dense(n_tags, activation="softmax"))(x)

#sentence level embeddings
#sent_embedding = Lambda(SentEmbedding, output_shape=(3, 768))(input_text)
'''
initializer = tf.keras.initializers.RandomUniform(minval=-tf.sqrt(3 / 30), maxval=tf.sqrt(3 / 30))
character_embedding = tf.keras.layers.Embedding(n_words2, 30, input_length=50, embeddings_initializer=initializer)
conv_net = character_embedding(tf.map_fn(fn=lambda t: words2index2.get(t), elems=input_text))
conv_net = tf.keras.layers.Conv1D(filters=64, kernel_size=3, activity_regularizer=tf.keras.regularizers.L2(1e-3), activation='relu')(conv_net)
conv_net = tf.keras.layers.Conv1D(filters=64, kernel_size=3, activity_regularizer=tf.keras.regularizers.L2(1e-3), activation='relu')(conv_net)
conv_net = tf.keras.layers.GlobalAveragePooling1D()(conv_net)
conv_net = tf.keras.layers.Dense(32, activation='relu')(conv_net)
'''
#so = Dense(512, activation="relu")(sent_embedding[0])
#concat = Concatenate(axis=-1)([wt, so])
#print(concat.shape)

#x = Bidirectional(LSTM(units=512, return_sequences=False, dropout=0.5, activity_regularizer=tf.keras.regularizers.L2(1e-3)))(tf.expand_dims(concat, axis=1, name=None))
                       
out = Dense(n_tags2, activation="softmax")(wt)

model = Model(input_text, out)
model.compile(optimizer=tf.compat.v1.keras.optimizers.RMSprop(learning_rate=0.01), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

model.summary()

#plot_model(model,'Elmo_tag.png',show_shapes= True)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
history = model.fit(np.array(X_tr), y_tr, validation_data=(np.array(X_val), y_val),
                    batch_size=batch_size, epochs=1, verbose=1, callbacks=[callback])
model.save("drive/MyDrive/dl_model")



Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 lambda_7 (Lambda)              (10, None, 1024)     0           ['input_9[0][0]']                
                                                                                                  
 bidirectional_14 (Bidirectiona  (10, None, 1024)    6295552     ['lambda_7[0][0]']               
 l)                                                                                               
                                                                                                  
 bidirectional_15 (Bidirectiona  (10, None, 1024)    6295552     ['bidirectional_14[0][0]'] 

Testing model:

In [ ]:
saved_model = tf.keras.models.load_model("drive/MyDrive/dl_model")
X_te, y_te = get_xy(dataset_df_test)

X_te = X_te[:-(len(X_te) % batch_size)]
test_pred = saved_model.predict(np.array(X_te), verbose=1, batch_size=batch_size)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


NameError: ignored

Output of results:

In [ ]:

idx2tag = {
    0: 'O',
    1: 'B-corporation',
    2: 'I-corporation',
    3: 'B-creative-work',
    4: 'I-creative-work',
    5: 'B-group',
    6: 'I-group',
    7: 'B-location',
    8: 'I-location',
    9: 'B-person',
    10: 'I-person',
    11: 'B-product',
    12: 'I-product'
} #this is for wnut2017

tags2index = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8} #this is for conll
idx2tag = {i: w for w, i in tags2index.items()}
print(idx2tag)
def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i])
        out.append(out_i)
    return out

def test2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            out_i.append(idx2tag[p])
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = test2label(y_te[:-(len(y_te) % batch_size)])

print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))

print(classification_report(test_labels, pred_labels))


{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC', 7: 'B-MISC', 8: 'I-MISC'}
F1-score: 87.0%
              precision    recall  f1-score   support

         LOC       0.91      0.91      0.91      7138
        MISC       0.74      0.76      0.75      3436
         ORG       0.80      0.85      0.83      6313
         PER       0.93      0.93      0.93      6584

   micro avg       0.86      0.88      0.87     23471
   macro avg       0.85      0.86      0.85     23471
weighted avg       0.86      0.88      0.87     23471



F1-score: 91.8%

            precision    recall  f1-score   support

         LOC       0.94      0.95      0.95      7138
        MISC       0.88      0.83      0.86      3436
         ORG       0.87      0.88      0.88      6313
         PER       0.95      0.96      0.96      6584

     micro avg       0.92      0.92      0.92     23471
    macro avg       0.91      0.91      0.91     23471
    weighted avg       0.92      0.92      0.92     23471

on CONLL 2003 with 2-stack bilstm; 10 batch size


F1-score: 41.8%
    
                precision    recall  f1-score   support

    corporation       0.00      0.00      0.00       221
    creative-work       0.00      0.00      0.00       140
    group       0.00      0.00      0.00       263
    location       0.37      0.54      0.44       547
    person       0.68      0.63      0.65       657
    product       0.00      0.00      0.00       142

    micro avg       0.50      0.36      0.42      1970
    macro avg       0.17      0.19      0.18      1970
    weighted avg       0.33      0.36      0.34      1970
  
on WNUT17, 2-stack bilstm, 20 batch size

F1-score: 40.0%
                        
                        precision    recall  f1-score   support

      corporation       0.36      0.04      0.07       221
    creative-work       0.00      0.00      0.00       140
            group       0.00      0.00      0.00       263
         location       0.42      0.37      0.39       547
           person       0.77      0.59      0.67       660
          product       0.00      0.00      0.00       142

        micro avg       0.59      0.30      0.40      1973
        macro avg       0.26      0.17      0.19      1973
     weighted avg       0.41      0.30      0.34      1973
 on WNUT17, 2-stack bilstm, 15 batch size

 F1-score: 45.1%
               
                    precision    recall  f1-score   support

      corporation       0.71      0.25      0.37       221
    creative-work       0.20      0.02      0.04       140
        group       0.40      0.01      0.01       263
     location       0.55      0.41      0.47       547
       person       0.74      0.61      0.67       660
      product       0.08      0.01      0.02       142

    micro avg       0.64      0.35      0.45      1973
    macro avg       0.45      0.22      0.26      1973
 weighted avg       0.55      0.35      0.40      1973

  on WNUT17, 2-stack bilstm, 10 batch size

F1-score: 50.7%
               precision    recall  f1-score   support

  corporation       0.69      0.27      0.38       221
creative-work       0.00      0.00      0.00       140
        group       0.48      0.16      0.23       263
     location       0.50      0.56      0.53       547
       person       0.79      0.65      0.71       660
      product       0.00      0.00      0.00       142

    micro avg       0.63      0.42      0.51      1973
    macro avg       0.41      0.27      0.31      1973
 weighted avg       0.55      0.42      0.46      1973
   on WNUT17, 10 batch size, 0.01 learning rate


